# 两个规则都是0.511

In [22]:
import gc
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, roc_auc_score
import datetime
import time
import lightgbm as lgb
import xgboost as xgb
import catboost as ctb
import numpy as np
import pandas as pd
import os
import math

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [23]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [24]:
train_sales  = pd.read_csv('train_sales_data.csv')
evaluation_public = pd.read_csv('evaluation_public.csv')
submit_example    = pd.read_csv('submit_example.csv')

In [25]:
day_map = {1:31,2:28,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
train_sales['daycount']=train_sales['regMonth'].map(day_map)
train_sales.loc[(train_sales.regMonth==2)&(train_sales.regYear==2016),'daycount']=29
#train_sales['salesVolume']/=train_sales['daycount']
train_sales.head(10)

,province,adcode,model,bodyType,regYear,regMonth,salesVolume,daycount
0,上海,310000,3c974920a76ac9c1,SUV,2016,1,292,31
1,云南,530000,3c974920a76ac9c1,SUV,2016,1,466,31
2,内蒙古,150000,3c974920a76ac9c1,SUV,2016,1,257,31
3,北京,110000,3c974920a76ac9c1,SUV,2016,1,408,31
4,四川,510000,3c974920a76ac9c1,SUV,2016,1,610,31
5,安徽,340000,3c974920a76ac9c1,SUV,2016,1,206,31
6,山东,370000,3c974920a76ac9c1,SUV,2016,1,503,31
7,山西,140000,3c974920a76ac9c1,SUV,2016,1,236,31
8,广东,440000,3c974920a76ac9c1,SUV,2016,1,3635,31
9,广西,450000,3c974920a76ac9c1,SUV,2016,1,450,31


In [26]:
train_sales2 = train_sales.set_index(["adcode", "bodyType", "model", "province","regYear", "regMonth"])[["salesVolume"]].unstack(level=-1)
train_sales2.head(10)

salesVolume             \
regMonth                                                    1    2     3    
adcode bodyType  model            province regYear                          
110000 Hatchback 3e21824be728cbec 北京       2016            315  100   199   
                                           2017             93   60    87   
                 7a7885e2d7c00bcf 北京       2016            817  426  1017   
                                           2017            471  523   831   
                 b4be3a4917289c82 北京       2016            175  106   251   
                                           2017             93  117   153   
       MPV       17bc272c93f19d56 北京       2016            456  125   304   
                                           2017            151   92   197   
                 79de4e4b24c35b04 北京       2016            920  174   284   
                                           2017            178  127   199   

                                                                             \
regMonth                                             4    5    6    7    8    
adcode bodyType  model            province regYear                            
110000 Hatchback 3e21824be728cbec 北京       2016     163  119  141  164  139   
                                           2017      42   47   47   40   41   
                 7a7885e2d7c00bcf 北京       2016     846  916  826  879  911   
                                           2017     698  714  664  617  559   
                 b4be3a4917289c82 北京       2016     267  259  192  172  203   
                                           2017     137  139   94   88  112   
       MPV       17bc272c93f19d56 北京       2016     246  278  163  179  202   
                                           2017     124  132  112  101   80   
                 79de4e4b24c35b04 北京       2016     352  376  194  240  313   
                                           2017     127  128  132  116  105   

                                                                           
regMonth                                              9    10    11    12  
adcode bodyType  model            province regYear                         
110000 Hatchback 3e21824be728cbec 北京       2016      203  168   222   242  
                                           2017       55   59    53    96  
                 7a7885e2d7c00bcf 北京       2016     1074  915  1162  1615  
                                           2017      655  482   726   739  
                 b4be3a4917289c82 北京       2016      232  200   230   226  
                                           2017      163  169   148   284  
       MPV       17bc272c93f19d56 北京       2016      242  199   305   514  
                                           2017      136  106   116   176  
                 79de4e4b24c35b04 北京       2016      370  330   446   852  
                                           2017       91   84   126   164

In [27]:
train_sales2.columns=['1','2','3','4','5','6','7','8','9','10','11','12']
train_sales2.head(10)

1    2     3    4    5  \
adcode bodyType  model            province regYear                             
110000 Hatchback 3e21824be728cbec 北京       2016     315  100   199  163  119   
                                           2017      93   60    87   42   47   
                 7a7885e2d7c00bcf 北京       2016     817  426  1017  846  916   
                                           2017     471  523   831  698  714   
                 b4be3a4917289c82 北京       2016     175  106   251  267  259   
                                           2017      93  117   153  137  139   
       MPV       17bc272c93f19d56 北京       2016     456  125   304  246  278   
                                           2017     151   92   197  124  132   
                 79de4e4b24c35b04 北京       2016     920  174   284  352  376   
                                           2017     178  127   199  127  128   

                                                      6    7    8     9   10  \
adcode bodyType  model            province regYear                             
110000 Hatchback 3e21824be728cbec 北京       2016     141  164  139   203  168   
                                           2017      47   40   41    55   59   
                 7a7885e2d7c00bcf 北京       2016     826  879  911  1074  915   
                                           2017     664  617  559   655  482   
                 b4be3a4917289c82 北京       2016     192  172  203   232  200   
                                           2017      94   88  112   163  169   
       MPV       17bc272c93f19d56 北京       2016     163  179  202   242  199   
                                           2017     112  101   80   136  106   
                 79de4e4b24c35b04 北京       2016     194  240  313   370  330   
                                           2017     132  116  105    91   84   

                                                      11    12  
adcode bodyType  model            province regYear              
110000 Hatchback 3e21824be728cbec 北京       2016      222   242  
                                           2017       53    96  
                 7a7885e2d7c00bcf 北京       2016     1162  1615  
                                           2017      726   739  
                 b4be3a4917289c82 北京       2016      230   226  
                                           2017      148   284  
       MPV       17bc272c93f19d56 北京       2016      305   514  
                                           2017      116   176  
                 79de4e4b24c35b04 北京       2016      446   852  
                                           2017      126   164

In [28]:
train_sales2["Mean"] = train_sales2.mean(axis=1)
train_sales2["Median"] = train_sales2.median(axis=1)
train_sales2.head(10)

1    2     3    4    5  \
adcode bodyType  model            province regYear                             
110000 Hatchback 3e21824be728cbec 北京       2016     315  100   199  163  119   
                                           2017      93   60    87   42   47   
                 7a7885e2d7c00bcf 北京       2016     817  426  1017  846  916   
                                           2017     471  523   831  698  714   
                 b4be3a4917289c82 北京       2016     175  106   251  267  259   
                                           2017      93  117   153  137  139   
       MPV       17bc272c93f19d56 北京       2016     456  125   304  246  278   
                                           2017     151   92   197  124  132   
                 79de4e4b24c35b04 北京       2016     920  174   284  352  376   
                                           2017     178  127   199  127  128   

                                                      6    7    8     9   10  \
adcode bodyType  model            province regYear                             
110000 Hatchback 3e21824be728cbec 北京       2016     141  164  139   203  168   
                                           2017      47   40   41    55   59   
                 7a7885e2d7c00bcf 北京       2016     826  879  911  1074  915   
                                           2017     664  617  559   655  482   
                 b4be3a4917289c82 北京       2016     192  172  203   232  200   
                                           2017      94   88  112   163  169   
       MPV       17bc272c93f19d56 北京       2016     163  179  202   242  199   
                                           2017     112  101   80   136  106   
                 79de4e4b24c35b04 北京       2016     194  240  313   370  330   
                                           2017     132  116  105    91   84   

                                                      11    12        Mean  \
adcode bodyType  model            province regYear                           
110000 Hatchback 3e21824be728cbec 北京       2016      222   242  181.250000   
                                           2017       53    96   60.000000   
                 7a7885e2d7c00bcf 北京       2016     1162  1615  950.333333   
                                           2017      726   739  639.916667   
                 b4be3a4917289c82 北京       2016      230   226  209.416667   
                                           2017      148   284  141.416667   
       MPV       17bc272c93f19d56 北京       2016      305   514  267.750000   
                                           2017      116   176  126.916667   
                 79de4e4b24c35b04 北京       2016      446   852  404.250000   
                                           2017      126   164  131.416667   

                                                        Median  
adcode bodyType  model            province regYear              
110000 Hatchback 3e21824be728cbec 北京       2016     168.000000  
                                           2017      55.000000  
                 7a7885e2d7c00bcf 北京       2016     915.000000  
                                           2017     655.000000  
                 b4be3a4917289c82 北京       2016     209.416667  
                                           2017     139.000000  
       MPV       17bc272c93f19d56 北京       2016     246.000000  
                                           2017     124.000000  
                 79de4e4b24c35b04 北京       2016     352.000000  
                                           2017     127.000000

In [29]:
train_sales2['Month_sum_1'] = train_sales2['1']
for i in range(2,12):
    train_sales2['Month_sum_' + str(i) ] =train_sales2['Month_sum_' + str(i-1) ]+ train_sales2[ str(i) ]
train_sales2.head()

1    2     3    4    5  \
adcode bodyType  model            province regYear                             
110000 Hatchback 3e21824be728cbec 北京       2016     315  100   199  163  119   
                                           2017      93   60    87   42   47   
                 7a7885e2d7c00bcf 北京       2016     817  426  1017  846  916   
                                           2017     471  523   831  698  714   
                 b4be3a4917289c82 北京       2016     175  106   251  267  259   

                                                      6    7    8     9   10  \
adcode bodyType  model            province regYear                             
110000 Hatchback 3e21824be728cbec 北京       2016     141  164  139   203  168   
                                           2017      47   40   41    55   59   
                 7a7885e2d7c00bcf 北京       2016     826  879  911  1074  915   
                                           2017     664  617  559   655  482   
                 b4be3a4917289c82 北京       2016     192  172  203   232  200   

                                                      11    12        Mean  \
adcode bodyType  model            province regYear                           
110000 Hatchback 3e21824be728cbec 北京       2016      222   242  181.250000   
                                           2017       53    96   60.000000   
                 7a7885e2d7c00bcf 北京       2016     1162  1615  950.333333   
                                           2017      726   739  639.916667   
                 b4be3a4917289c82 北京       2016      230   226  209.416667   

                                                        Median  Month_sum_1  \
adcode bodyType  model            province regYear                            
110000 Hatchback 3e21824be728cbec 北京       2016     168.000000          315   
                                           2017      55.000000           93   
                 7a7885e2d7c00bcf 北京       2016     915.000000          817   
                                           2017     655.000000          471   
                 b4be3a4917289c82 北京       2016     209.416667          175   

                                                    Month_sum_2  Month_sum_3  \
adcode bodyType  model            province regYear                             
110000 Hatchback 3e21824be728cbec 北京       2016             415          614   
                                           2017             153          240   
                 7a7885e2d7c00bcf 北京       2016            1243         2260   
                                           2017             994         1825   
                 b4be3a4917289c82 北京       2016             281          532   

                                                    Month_sum_4  Month_sum_5  \
adcode bodyType  model            province regYear                             
110000 Hatchback 3e21824be728cbec 北京       2016             777          896   
                                           2017             282          329   
                 7a7885e2d7c00bcf 北京       2016            3106         4022   
                                           2017            2523         3237   
                 b4be3a4917289c82 北京       2016             799         1058   

                                                    Month_sum_6  Month_sum_7  \
adcode bodyType  model            province regYear                             
110000 Hatchback 3e21824be728cbec 北京       2016            1037         1201   
                                           2017             376          416   
                 7a7885e2d7c00bcf 北京       2016            4848         5727   
                                           2017            3901         4518   
                 b4be3a4917289c82 北京       2016            1250         1422   

                                                    Month_sum_8  Month_sum_9  \
adcode bodyType  model            province regYear                  

In [30]:
tmp = train_sales2

In [31]:
#一阶差分，一阶比值
for i in range(2,13):
    thismonth = tmp[str(i)]
    lastmonth = tmp[str(i-1)]
    tmp['yijiechafen_' + str(i)] = thismonth - lastmonth
    tmp['yijiebizhi_' + str(i)] = thismonth / lastmonth

In [35]:
#二阶差分，二阶比值
for i in range(3,13):
    thismonth = tmp['yijiechafen_'+str(i)]
    lastmonth = tmp['yijiechafen_'+str(i-1)]
    tmp['erjiechafen_' + str(i)] = thismonth - lastmonth
    thismonth = tmp['yijiebizhi_'+str(i)]
    lastmonth = tmp['yijiebizhi_'+str(i-1)]
    tmp['erjiebizhi_' + str(i)] = thismonth / lastmonth

In [43]:
tmp.head()

1_2016  1_2017  2_2016  2_2017  \
adcode bodyType  model            province                                   
110000 Hatchback 3e21824be728cbec 北京           315      93     100      60   
                 7a7885e2d7c00bcf 北京           817     471     426     523   
                 b4be3a4917289c82 北京           175      93     106     117   
       MPV       17bc272c93f19d56 北京           456     151     125      92   
                 79de4e4b24c35b04 北京           920     178     174     127   

                                            3_2016  3_2017  4_2016  4_2017  \
adcode bodyType  model            province                                   
110000 Hatchback 3e21824be728cbec 北京           199      87     163      42   
                 7a7885e2d7c00bcf 北京          1017     831     846     698   
                 b4be3a4917289c82 北京           251     153     267     137   
       MPV       17bc272c93f19d56 北京           304     197     246     124   
                 79de4e4b24c35b04 北京           284     199     352     127   

                                            5_2016  5_2017  6_2016  6_2017  \
adcode bodyType  model            province                                   
110000 Hatchback 3e21824be728cbec 北京           119      47     141      47   
                 7a7885e2d7c00bcf 北京           916     714     826     664   
                 b4be3a4917289c82 北京           259     139     192      94   
       MPV       17bc272c93f19d56 北京           278     132     163     112   
                 79de4e4b24c35b04 北京           376     128     194     132   

                                            7_2016  7_2017  8_2016  8_2017  \
adcode bodyType  model            province                                   
110000 Hatchback 3e21824be728cbec 北京           164      40     139      41   
                 7a7885e2d7c00bcf 北京           879     617     911     559   
                 b4be3a4917289c82 北京           172      88     203     112   
       MPV       17bc272c93f19d56 北京           179     101     202      80   
                 79de4e4b24c35b04 北京           240     116     313     105   

                                            9_2016  9_2017  10_2016  10_2017  \
adcode bodyType  model            province                                     
110000 Hatchback 3e21824be728cbec 北京           203      55      168       59   
                 7a7885e2d7c00bcf 北京          1074     655      915      482   
                 b4be3a4917289c82 北京           232     163      200      169   
       MPV       17bc272c93f19d56 北京           242     136      199      106   
                 79de4e4b24c35b04 北京           370      91      330       84   

                                            11_2016  11_2017  12_2016  \
adcode bodyType  model            province                              
110000 Hatchback 3e21824be728cbec 北京            222       53      242   
                 7a7885e2d7c00bcf 北京           1162      726     1615   
                 b4be3a4917289c82 北京            230      148      226   
       MPV       17bc272c93f19d56 北京            305      116      514   
                 79de4e4b24c35b04 北京            446      126      852   

                                            12_2017   Mean_2016   Mean_2017  \
adcode bodyType  model            province                                    
110000 Hatchback 3e21824be728cbec 北京             96  181.250000   60.000000   
                 7a7885e2d7c00bcf 北京            739  950.333333  639.916667   
                 b4be3a4917289c82 北京            284  209.416667  141.416667   
       MPV       17bc272c93f19d56 北京            176  267.750000  126.916667   
                 79de4e4b24c35b04 北京            164  404.250000  131.416667   

                                            Median_2016  Median_2017  \
adcode bodyType  model            province                             
110000 Hatchback 3e21824be728cbec 北京         168.000000         55.0   
             

In [37]:
train_sales2 = tmp

In [38]:
train_sales2['quarter1_mean']=(train_sales2['1']+train_sales2['2']+train_sales2['3'])/3
train_sales2['quarter2_mean']=(train_sales2['4']+train_sales2['5']+train_sales2['6'])/3
train_sales2['quarter3_mean']=(train_sales2['7']+train_sales2['8']+train_sales2['9'])/3
train_sales2['quarter4_mean']=(train_sales2['10']+train_sales2['11']+train_sales2['12'])/3
train_sales2=train_sales2.unstack("regYear")
train_sales2.head(10)

1          2           3        \
regYear                                     2016  2017 2016  2017  2016  2017   
adcode bodyType  model            province                                      
110000 Hatchback 3e21824be728cbec 北京         315    93  100    60   199    87   
                 7a7885e2d7c00bcf 北京         817   471  426   523  1017   831   
                 b4be3a4917289c82 北京         175    93  106   117   251   153   
       MPV       17bc272c93f19d56 北京         456   151  125    92   304   197   
                 79de4e4b24c35b04 北京         920   178  174   127   284   199   
                 7aab7fca2470987e 北京         406   424  155   244   342   378   
                 c06a2a387c0ee510 北京        1071  1352  361  1114   867  1508   
                 f270f6a489c6a9d7 北京         409   106  156   118   229   157   
       SUV       17363f08d683d52b 北京         580   124  162    81   255    65   
                 2a2ab41f8f6ff1cb 北京         458   212   72   102   162   180   

                                              4          5          6        \
regYear                                    2016  2017 2016  2017 2016  2017   
adcode bodyType  model            province                                    
110000 Hatchback 3e21824be728cbec 北京        163    42  119    47  141    47   
                 7a7885e2d7c00bcf 北京        846   698  916   714  826   664   
                 b4be3a4917289c82 北京        267   137  259   139  192    94   
       MPV       17bc272c93f19d56 北京        246   124  278   132  163   112   
                 79de4e4b24c35b04 北京        352   127  376   128  194   132   
                 7aab7fca2470987e 北京        385   320  371   244  412   260   
                 c06a2a387c0ee510 北京        872  1132  926  1430  824  1670   
                 f270f6a489c6a9d7 北京        233   120  214   110  173   104   
       SUV       17363f08d683d52b 北京        224    48  215    29  203    26   
                 2a2ab41f8f6ff1cb 北京        260   142  324   124  331    93   

                                               7          8           9        \
regYear                                     2016  2017 2016  2017  2016  2017   
adcode bodyType  model            province                                      
110000 Hatchback 3e21824be728cbec 北京         164    40  139    41   203    55   
                 7a7885e2d7c00bcf 北京         879   617  911   559  1074   655   
                 b4be3a4917289c82 北京         172    88  203   112   232   163   
       MPV       17bc272c93f19d56 北京         179   101  202    80   242   136   
                 79de4e4b24c35b04 北京         240   116  313   105   370    91   
                 7aab7fca2470987e 北京         397   250  450   268   433   314   
                 c06a2a387c0ee510 北京        1014  1367  976  1416  1027  1463   
                 f270f6a489c6a9d7 北京         229    70  238    81   282    80   
       SUV       17363f08d683d52b 北京         267    33  229    15   259    60   
                 2a2ab41f8f6ff1cb 北京         345   130  369   133   502   130   

                                             10          11          12        \
regYear                                    2016  2017  2016  2017  2016  2017   
adcode bodyType  model            province                                      
110000 Hatchback 3e21824be728cbec 北京        168    59   222    53   242    96   
                 7a7885e2d7c00bcf 北京        915   482  1162   726  1615   739   
                 b4be3a4917289c82 北京        200   169   230   148   226   284   
       MPV       17bc272c93f19d56 北京        199   106   305   116   514   176   
                 79de4e4b24c35b04 北京        330    84   446   126   852   164   
                 7aab7fca2470987e 北京        353   181   372   257   511   216   
                 c06a2a387c0ee510 北京        677  1056   793  1425  1627  1358   
                 f270f6a489c6a9d7 北京        198    59   305    99   504   321   
       SUV       173

In [39]:
train_sales2.columns= ["_".join([str(i) for i in x]) for x in train_sales2.columns.ravel()]
train_sales2.head(10)

1_2016  1_2017  2_2016  2_2017  \
adcode bodyType  model            province                                   
110000 Hatchback 3e21824be728cbec 北京           315      93     100      60   
                 7a7885e2d7c00bcf 北京           817     471     426     523   
                 b4be3a4917289c82 北京           175      93     106     117   
       MPV       17bc272c93f19d56 北京           456     151     125      92   
                 79de4e4b24c35b04 北京           920     178     174     127   
                 7aab7fca2470987e 北京           406     424     155     244   
                 c06a2a387c0ee510 北京          1071    1352     361    1114   
                 f270f6a489c6a9d7 北京           409     106     156     118   
       SUV       17363f08d683d52b 北京           580     124     162      81   
                 2a2ab41f8f6ff1cb 北京           458     212      72     102   

                                            3_2016  3_2017  4_2016  4_2017  \
adcode bodyType  model            province                                   
110000 Hatchback 3e21824be728cbec 北京           199      87     163      42   
                 7a7885e2d7c00bcf 北京          1017     831     846     698   
                 b4be3a4917289c82 北京           251     153     267     137   
       MPV       17bc272c93f19d56 北京           304     197     246     124   
                 79de4e4b24c35b04 北京           284     199     352     127   
                 7aab7fca2470987e 北京           342     378     385     320   
                 c06a2a387c0ee510 北京           867    1508     872    1132   
                 f270f6a489c6a9d7 北京           229     157     233     120   
       SUV       17363f08d683d52b 北京           255      65     224      48   
                 2a2ab41f8f6ff1cb 北京           162     180     260     142   

                                            5_2016  5_2017  6_2016  6_2017  \
adcode bodyType  model            province                                   
110000 Hatchback 3e21824be728cbec 北京           119      47     141      47   
                 7a7885e2d7c00bcf 北京           916     714     826     664   
                 b4be3a4917289c82 北京           259     139     192      94   
       MPV       17bc272c93f19d56 北京           278     132     163     112   
                 79de4e4b24c35b04 北京           376     128     194     132   
                 7aab7fca2470987e 北京           371     244     412     260   
                 c06a2a387c0ee510 北京           926    1430     824    1670   
                 f270f6a489c6a9d7 北京           214     110     173     104   
       SUV       17363f08d683d52b 北京           215      29     203      26   
                 2a2ab41f8f6ff1cb 北京           324     124     331      93   

                                            7_2016  7_2017  8_2016  8_2017  \
adcode bodyType  model            province                                   
110000 Hatchback 3e21824be728cbec 北京           164      40     139      41   
                 7a7885e2d7c00bcf 北京           879     617     911     559   
                 b4be3a4917289c82 北京           172      88     203     112   
       MPV       17bc272c93f19d56 北京           179     101     202      80   
                 79de4e4b24c35b04 北京           240     116     313     105   
                 7aab7fca2470987e 北京           397     250     450     268   
                 c06a2a387c0ee510 北京          1014    1367     976    1416   
                 f270f6a489c6a9d7 北京           229      70     238      81   
       SUV       17363f08d683d52b 北京           267      33     229      15   
                 2a2ab41f8f6ff1cb 北京           345     130     369     133   

                                            9_2016  9_2017  10_2016  10_2017  \
adcode bodyType  model            province                                     
110000 Hatchback 3e21824be728cbec 北京           203      55      168       59   
                 7a7885e2d7c00bcf 北京          1074     6

In [64]:
tmp =train_sales2

In [45]:
for i in range(1,13):
    tmp['linnianchafen_'+str(i)]= tmp[str(i)+'_2017']-tmp[str(i)+'_2016']
    tmp['linnianbizhi_'+str(i)]= tmp[str(i)+'_2017']/tmp[str(i)+'_2016']

In [47]:

train_sales2=tmp

In [67]:
#train_sales2 = train_sales2.reset_index()
#train_sales2.head(10)
tmp.head()

,adcode,bodyType,model,province,Mean_2016,Mean_2017,Median_2016,Median_2017,Month_sum_1_2016,Month_sum_1_2017,Month_sum_2_2016,Month_sum_2_2017,Month_sum_3_2016,Month_sum_3_2017,Month_sum_4_2016,Month_sum_4_2017,Month_sum_5_2016,Month_sum_5_2017,Month_sum_6_2016,Month_sum_6_2017,Month_sum_7_2016,Month_sum_7_2017,Month_sum_8_2016,Month_sum_8_2017,Month_sum_9_2016,Month_sum_9_2017,Month_sum_10_2016,Month_sum_10_2017,Month_sum_11_2016,Month_sum_11_2017,yijiechafen_2_2016,yijiechafen_2_2017,yijiebizhi_2_2016,yijiebizhi_2_2017,yijiechafen_3_2016,yijiechafen_3_2017,yijiebizhi_3_2016,yijiebizhi_3_2017,yijiechafen_4_2016,yijiechafen_4_2017,yijiebizhi_4_2016,yijiebizhi_4_2017,yijiechafen_5_2016,yijiechafen_5_2017,yijiebizhi_5_2016,yijiebizhi_5_2017,yijiechafen_6_2016,yijiechafen_6_2017,yijiebizhi_6_2016,yijiebizhi_6_2017,yijiechafen_7_2016,yijiechafen_7_2017,yijiebizhi_7_2016,yijiebizhi_7_2017,yijiechafen_8_2016,yijiechafen_8_2017,yijiebizhi_8_2016,yijiebizhi_8_2017,yijiechafen_9_2016,yijiechafen_9_2017,yijiebizhi_9_2016,yijiebizhi_9_2017,yijiechafen_10_2016,yijiechafen_10_2017,yijiebizhi_10_2016,yijiebizhi_10_2017,yijiechafen_11_2016,yijiechafen_11_2017,yijiebizhi_11_2016,yijiebizhi_11_2017,yijiechafen_12_2016,yijiechafen_12_2017,yijiebizhi_12_2016,yijiebizhi_12_2017,erjiechafen_3_2016,erjiechafen_3_2017,erjiebizhi_3_2016,erjiebizhi_3_2017,erjiechafen_4_2016,erjiechafen_4_2017,erjiebizhi_4_2016,erjiebizhi_4_2017,erjiechafen_5_2016,erjiechafen_5_2017,erjiebizhi_5_2016,erjiebizhi_5_2017,erjiechafen_6_2016,erjiechafen_6_2017,erjiebizhi_6_2016,erjiebizhi_6_2017,erjiechafen_7_2016,erjiechafen_7_2017,erjiebizhi_7_2016,erjiebizhi_7_2017,erjiechafen_8_2016,erjiechafen_8_2017,erjiebizhi_8_2016,erjiebizhi_8_2017,erjiechafen_9_2016,erjiechafen_9_2017,erjiebizhi_9_2016,erjiebizhi_9_2017,erjiechafen_10_2016,erjiechafen_10_2017,erjiebizhi_10_2016,erjiebizhi_10_2017,erjiechafen_11_2016,erjiechafen_11_2017,erjiebizhi_11_2016,erjiebizhi_11_2017,erjiechafen_12_2016,erjiechafen_12_2017,erjiebizhi_12_2016,erjiebizhi_12_2017,quarter1_mean_2016,quarter1_mean_2017,quarter2_mean_2016,quarter2_mean_2017,quarter3_mean_2016,quarter3_mean_2017,quarter4_mean_2016,quarter4_mean_2017,linnianchafen_1,linnianchafen_2,linnianchafen_3,linnianchafen_4,linnianchafen_5,linnianchafen_6,linnianchafen_7,linnianchafen_8,linnianchafen_9,linnianchafen_10,linnianchafen_11,linnianchafen_12,linnianbizhi_1,linnianbizhi_2,linnianbizhi_3,linnianbizhi_4,linnianbizhi_5,linnianbizhi_6,linnianbizhi_7,linnianbizhi_8,linnianbizhi_9,linnianbizhi_10,linnianbizhi_11,linnianbizhi_12
0,110000,Hatchback,3e21824be728cbec,北京,181.250000,60.000000,168.000000,55.0,315,93,415,153,614,240,777,282,896,329,1037,376,1201,416,1340,457,1543,512,1711,571,1933,624,-215,-33,0.317460,0.645161,99,27,1.990000,1.450000,-36,-45,0.819095,0.482759,-44,5,0.730061,1.119048,22,0,1.184874,1.000000,23,-7,1.163121,0.851064,-25,1,0.847561,1.025000,64,14,1.460432,1.341463,-35,4,0.827586,1.072727,54,-6,1.321429,0.898305,20,43,1.090090,1.811321,314,60,6.268500,2.247500,-135,-72,0.411606,0.332937,-8,50,0.891302,2.318027,66,-5,1.622979,0.893617,1,-7,0.981641,0.851064,-48,8,0.728696,1.204375,89,13,1.723099,1.308745,-99,-10,0.566672,0.799669,89,-10,1.596726,0.837403,-34,49,0.824933,2.016376,204.666667,80.000000,141.000000,45.333333,168.666667,45.333333,210.666667,69.333333,-222,-40,-112,-121,-72,-94,-124,-98,-148,-109,-169,-146,0.295238,0.600000,0.437186,0.257669,0.394958,0.333333,0.243902,0.294964,0.270936,0.351190,0.238739,0.396694
1,110000,Hatchback,7a7885e2d7c00bcf,北京,950.333333,639.916667,915.000000,655.0,817,471,1243,994,2260,1825,3106,2523,4022,3237,4848,3901,5727,4518,6638,5077,7712,5732,8627,6214,9789,6940,-391,52,0.521420,1.110403,591,308,2.387324,1.588910,-171,-133,0.831858,0.839952,70,16,1.082742,1.022923,-90,-50,0.901747,0.929972,53,-47,1.064165,0.929217,32,-58,1.036405,0.905997,163,96,1.178924,1.171735,-159,-173,0.851955,0.735878,247,244,1.269945,1.506224,453,13,1.389845,1.017906,982,256,4.578506,1.430931,-762,-441,0

In [68]:
# for i in range(1,13):
#     del tmp[str(i)+'_2016'],tmp[str(i)+'_2017']
    
tmp.to_csv('sales_feats.csv',index = False)

    
    
    
    
    

In [57]:
tezheng1

NameError: name 'tezheng1' is not defined

In [11]:
train_sales2['20171_divide_20161']=train_sales2['1_2017']/train_sales2['1_2016']
train_sales2['20172_divide_20162']=train_sales2['2_2017']/train_sales2['2_2016']
train_sales2['20173_divide_20163']=train_sales2['3_2017']/train_sales2['3_2016']
train_sales2['20174_divide_20164']=train_sales2['4_2017']/train_sales2['4_2016']

train_sales2['2017y_divide_2016y_mean']=train_sales2['Mean_2017']/train_sales2['Mean_2016']
train_sales2['2017y_divide_2016y_median']=train_sales2['Median_2017']/train_sales2['Median_2016']
train_sales2['2017q1_divide_2016q1']=train_sales2['quarter1_mean_2017']/train_sales2['quarter1_mean_2016']
train_sales2['2017q2_divide_2016q2']=train_sales2['quarter2_mean_2017']/train_sales2['quarter2_mean_2016']

qushim1=(0.25*train_sales2['2017y_divide_2016y_mean']+0.25*train_sales2['2017y_divide_2016y_mean'])+0.3*train_sales2['2017q1_divide_2016q1']+0.2*train_sales2['20171_divide_20161']
qushim2=(0.25*train_sales2['2017y_divide_2016y_mean']+0.25*train_sales2['2017y_divide_2016y_mean'])+0.3*train_sales2['2017q1_divide_2016q1']+0.2*train_sales2['20172_divide_20162']
qushim3=(0.25*train_sales2['2017y_divide_2016y_mean']+0.25*train_sales2['2017y_divide_2016y_mean'])+0.3*train_sales2['2017q1_divide_2016q1']+0.2*train_sales2['20173_divide_20163']
qushim4=(0.25*train_sales2['2017y_divide_2016y_mean']+0.25*train_sales2['2017y_divide_2016y_mean'])+0.3*train_sales2['2017q2_divide_2016q2']+0.2*train_sales2['20174_divide_20164']


qushi1_v_2016=train_sales2['1_2016'].values* qushim1* qushim1
qushi2_v_2016=train_sales2['2_2016'].values* qushim2* qushim2
qushi3_v_2016=train_sales2['3_2016'].values* qushim3* qushim3
qushi4_v_2016=train_sales2['4_2016'].values* qushim4* qushim4

qushi1_v_2017=train_sales2['1_2017'].values* qushim1
qushi2_v_2017=train_sales2['2_2017'].values* qushim2
qushi3_v_2017=train_sales2['3_2017'].values* qushim3
qushi4_v_2017=train_sales2['4_2017'].values* qushim4

qushi1_v=qushi1_v_2016*0.3+qushi1_v_2017*0.7
qushi2_v=qushi2_v_2016*0.3+qushi2_v_2017*0.7
qushi3_v=qushi3_v_2016*0.3+qushi3_v_2017*0.7
qushi4_v=qushi4_v_2016*0.3+qushi4_v_2017*0.7

train_sales2['pre1']=qushi1_v
train_sales2['pre2']=qushi2_v
train_sales2['pre3']=qushi3_v
train_sales2['pre4']=qushi4_v

In [12]:
train_sales2.head(10)

,adcode,bodyType,model,province,1_2016,1_2017,2_2016,2_2017,3_2016,3_2017,...,20173_divide_20163,20174_divide_20164,2017y_divide_2016y_mean,2017y_divide_2016y_median,2017q1_divide_2016q1,2017q2_divide_2016q2,pre1,pre2,pre3,pre4
0,110000,Hatchback,3e21824be728cbec,北京,10.161290,3.000000,3.448276,2.142857,6.419355,2.806452,...,0.437186,0.257669,0.332244,0.337423,0.396892,0.320840,1.084129,0.787664,0.999430,0.468247
1,110000,Hatchback,7a7885e2d7c00bcf,北京,26.354839,15.193548,14.689655,18.678571,32.806452,26.806452,...,0.817109,0.825059,0.676817,0.739709,0.821636,0.802426,11.323318,14.076295,19.553208,16.804467
2,110000,Hatchback,b4be3a4917289c82,北京,5.645161,3.000000,3.655172,4.178571,8.096774,4.935484,...,0.609562,0.513109,0.677223,0.665932,0.696326,0.515069,2.096873,2.930802,3.401254,2.852070
3,110000,MPV,17bc272c93f19d56,北京,14.709677,4.870968,4.310345,3.285714,9.806452,6.354839,...,0.648026,0.504065,0.476143,0.504065,0.503409,0.536467,2.467377,1.624806,3.098901,2.060727
4,110000,MPV,79de4e4b24c35b04,北京,29.677419,5.741935,6.000000,4.535714,9.161290,6.419355,...,0.700704,0.360795,0.327291,0.360795,0.372379,0.420797,2.140426,1.681795,2.341551,1.534242
5,110000,MPV,7aab7fca2470987e,北京,13.096774,13.677419,5.344828,8.714286,11.032258,12.193548,...,1.105263,0.831169,0.735421,0.676742,1.173421,0.705974,12.278659,8.133256,10.959440,7.709241
6,110000,MPV,c06a2a387c0ee510,北京,34.548387,43.612903,12.448276,39.785714,27.967742,48.645161,...,1.739331,1.298165,1.485182,1.493883,1.761420,1.614839,70.567023,66.827098,77.114795,58.541049
7,110000,MPV,f270f6a489c6a9d7,北京,13.193548,3.419355,5.379310,4.214286,7.387097,5.064516,...,0.685590,0.515021,0.451506,0.451541,0.489144,0.538988,1.728327,2.012999,2.382207,1.933739
8,110000,SUV,17363f08d683d52b,北京,18.709677,4.000000,5.586207,2.892857,8.225806,2.096774,...,0.254902,0.214286,0.209807,0.214286,0.276420,0.160715,0.944090,0.732392,0.491247,0.305523
9,110000,SUV,2a2ab41f8f6ff1cb,北京,14.774194,6.838710,2.482759,3.642857,5.225806,5.806452,...,1.111111,0.546154,0.373281,0.365126,0.724467,0.392461,3.469925,2.140748,3.159971,1.815217


In [14]:
presale1 = train_sales2[["adcode", "bodyType", "model", "province",'pre1']].rename(columns={'pre1':'forecastVolum1'})
presale1.head(10)

,adcode,bodyType,model,province,forecastVolum1
0,110000,Hatchback,3e21824be728cbec,北京,1.084129
1,110000,Hatchback,7a7885e2d7c00bcf,北京,11.323318
2,110000,Hatchback,b4be3a4917289c82,北京,2.096873
3,110000,MPV,17bc272c93f19d56,北京,2.467377
4,110000,MPV,79de4e4b24c35b04,北京,2.140426
5,110000,MPV,7aab7fca2470987e,北京,12.278659
6,110000,MPV,c06a2a387c0ee510,北京,70.567023
7,110000,MPV,f270f6a489c6a9d7,北京,1.728327
8,110000,SUV,17363f08d683d52b,北京,0.944090
9,110000,SUV,2a2ab41f8f6ff1cb,北京,3.469925


In [16]:
presale1['regMonth'] = 1
presale2 = train_sales2[["adcode", "bodyType", "model", "province", 'pre2']].rename(columns={'pre2':'forecastVolum1'})
presale2['regMonth'] = 2
presale3 = train_sales2[["adcode", "bodyType", "model", "province", 'pre3']].rename(columns={'pre3':'forecastVolum1'})
presale3['regMonth'] = 3
presale4 = train_sales2[["adcode", "bodyType", "model", "province", 'pre4']].rename(columns={'pre4':'forecastVolum1'})
presale4['regMonth'] = 4

In [17]:
result=pd.concat([presale1,presale2,presale3,presale4]).reset_index(drop=True)
result.head(10)

,adcode,bodyType,forecastVolum1,model,province,regMonth
0,110000,Hatchback,1.084129,3e21824be728cbec,北京,NaN
1,110000,Hatchback,11.323318,7a7885e2d7c00bcf,北京,NaN
2,110000,Hatchback,2.096873,b4be3a4917289c82,北京,NaN
3,110000,MPV,2.467377,17bc272c93f19d56,北京,NaN
4,110000,MPV,2.140426,79de4e4b24c35b04,北京,NaN
5,110000,MPV,12.278659,7aab7fca2470987e,北京,NaN
6,110000,MPV,70.567023,c06a2a387c0ee510,北京,NaN
7,110000,MPV,1.728327,f270f6a489c6a9d7,北京,NaN
8,110000,SUV,0.944090,17363f08d683d52b,北京,NaN
9,110000,SUV,3.469925,2a2ab41f8f6ff1cb,北京,NaN


In [18]:
def rule1(train_sales):
    train_sales = train_sales.set_index(["adcode", "bodyType", "model", "province","regYear", "regMonth"])[["salesVolume"]].unstack(level=-1)
    train_sales.columns=['1','2','3','4','5','6','7','8','9','10','11','12']
    train_sales["Mean"] = train_sales.mean(axis=1)
    train_sales["Median"] = train_sales.median(axis=1)

    train_sales['quarter1_mean']=(train_sales['1']+train_sales['2']+train_sales['3'])/3
    train_sales['quarter2_mean']=(train_sales['4']+train_sales['5']+train_sales['6'])/3

    train_sales=train_sales.unstack("regYear")
    train_sales.columns= ["_".join([str(i) for i in x]) for x in train_sales.columns.ravel()]
    train_sales = train_sales.reset_index()
    train_sales['20171_divide_20161']=train_sales['1_2017']/train_sales['1_2016']
    train_sales['20172_divide_20162']=train_sales['2_2017']/train_sales['2_2016']
    train_sales['20173_divide_20163']=train_sales['3_2017']/train_sales['3_2016']
    train_sales['20174_divide_20164']=train_sales['4_2017']/train_sales['4_2016']

    train_sales['2017y_divide_2016y_mean']=train_sales['Mean_2017']/train_sales['Mean_2016']
    train_sales['2017y_divide_2016y_median']=train_sales['Median_2017']/train_sales['Median_2016']
    train_sales['2017q1_divide_2016q1']=train_sales['quarter1_mean_2017']/train_sales['quarter1_mean_2016']
    train_sales['2017q2_divide_2016q2']=train_sales['quarter2_mean_2017']/train_sales['quarter2_mean_2016']

    qushim1=(0.25*train_sales['2017y_divide_2016y_mean']+0.25*train_sales['2017y_divide_2016y_mean'])+0.3*train_sales['2017q1_divide_2016q1']+0.2*train_sales['20171_divide_20161']
    qushim2=(0.25*train_sales['2017y_divide_2016y_mean']+0.25*train_sales['2017y_divide_2016y_mean'])+0.3*train_sales['2017q1_divide_2016q1']+0.2*train_sales['20172_divide_20162']
    qushim3=(0.25*train_sales['2017y_divide_2016y_mean']+0.25*train_sales['2017y_divide_2016y_mean'])+0.3*train_sales['2017q1_divide_2016q1']+0.2*train_sales['20173_divide_20163']
    qushim4=(0.25*train_sales['2017y_divide_2016y_mean']+0.25*train_sales['2017y_divide_2016y_mean'])+0.3*train_sales['2017q2_divide_2016q2']+0.2*train_sales['20174_divide_20164']


    qushi1_v_2016=train_sales['1_2016'].values* qushim1* qushim1
    qushi2_v_2016=train_sales['2_2016'].values* qushim2* qushim2
    qushi3_v_2016=train_sales['3_2016'].values* qushim3* qushim3
    qushi4_v_2016=train_sales['4_2016'].values* qushim4* qushim4

    qushi1_v_2017=train_sales['1_2017'].values* qushim1
    qushi2_v_2017=train_sales['2_2017'].values* qushim2
    qushi3_v_2017=train_sales['3_2017'].values* qushim3
    qushi4_v_2017=train_sales['4_2017'].values* qushim4

    qushi1_v=qushi1_v_2016*0.3+qushi1_v_2017*0.7
    qushi2_v=qushi2_v_2016*0.3+qushi2_v_2017*0.7
    qushi3_v=qushi3_v_2016*0.3+qushi3_v_2017*0.7
    qushi4_v=qushi4_v_2016*0.3+qushi4_v_2017*0.7

    train_sales['pre1']=qushi1_v
    train_sales['pre2']=qushi2_v
    train_sales['pre3']=qushi3_v
    train_sales['pre4']=qushi4_v
    presale1 = train_sales[["adcode", "bodyType", "model", "province",'pre1']].rename(columns={'pre1':'forecastVolum1'})
    presale1['regMonth']=1
    presale2 = train_sales[["adcode", "bodyType", "model", "province", 'pre2']].rename(columns={'pre2':'forecastVolum1'})
    presale2['regMonth'] = 2
    presale3 = train_sales[["adcode", "bodyType", "model", "province", 'pre3']].rename(columns={'pre3':'forecastVolum1'})
    presale3['regMonth'] = 3
    presale4 = train_sales[["adcode", "bodyType", "model", "province", 'pre4']].rename(columns={'pre4':'forecastVolum1'})
    presale4['regMonth'] = 4

    result=pd.concat([presale1,presale2,presale3,presale4]).reset_index(drop=True)

    return result

rule1=rule1(train_sales.copy())

In [19]:
rule1.head(10)

,adcode,bodyType,model,province,forecastVolum1,regMonth
0,110000,Hatchback,3e21824be728cbec,北京,1.084129,1
1,110000,Hatchback,7a7885e2d7c00bcf,北京,11.323318,1
2,110000,Hatchback,b4be3a4917289c82,北京,2.096873,1
3,110000,MPV,17bc272c93f19d56,北京,2.467377,1
4,110000,MPV,79de4e4b24c35b04,北京,2.140426,1
5,110000,MPV,7aab7fca2470987e,北京,12.278659,1
6,110000,MPV,c06a2a387c0ee510,北京,70.567023,1
7,110000,MPV,f270f6a489c6a9d7,北京,1.728327,1
8,110000,SUV,17363f08d683d52b,北京,0.944090,1
9,110000,SUV,2a2ab41f8f6ff1cb,北京,3.469925,1


In [20]:
def yulaorule(train_sales):
    m1_12 = train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 1), 'salesVolume'].values / \
            train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 12), 'salesVolume'].values
    m1_11 = train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 1), 'salesVolume'].values / \
            train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 11), 'salesVolume'].values
    m1_10 = train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 1), 'salesVolume'].values / \
            train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 10), 'salesVolume'].values
    m1_09 = train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 1), 'salesVolume'].values / \
            train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 9), 'salesVolume'].values
    m1_08 = train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 1), 'salesVolume'].values / \
            train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 8), 'salesVolume'].values

    m1_12_volum = train_sales.loc[
                      (train_sales.regYear == 2017) & (train_sales.regMonth == 12), 'salesVolume'].values * m1_12
    m1_11_volum = train_sales.loc[
                      (train_sales.regYear == 2017) & (train_sales.regMonth == 11), 'salesVolume'].values * m1_11
    m1_10_volum = train_sales.loc[
                      (train_sales.regYear == 2017) & (train_sales.regMonth == 10), 'salesVolume'].values * m1_10
    m1_09_volum = train_sales.loc[
                      (train_sales.regYear == 2017) & (train_sales.regMonth == 9), 'salesVolume'].values * m1_09
    m1_08_volum = train_sales.loc[
                      (train_sales.regYear == 2017) & (train_sales.regMonth == 8), 'salesVolume'].values * m1_08

    evaluation_public.loc[
        evaluation_public.regMonth == 1, 'forecastVolum'] = m1_12_volum / 2 + m1_11_volum / 4 + m1_10_volum / 8 + m1_09_volum / 16 + m1_08_volum / 16

    # 2018年1、2、3月，提取方式历史月份销量比例，考虑时间衰减，月份越近占比越高
    m16_1_2 = train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 1), 'salesVolume'].values / \
              train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 2), 'salesVolume'].values
    m16_1_3 = train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 1), 'salesVolume'].values / \
              train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 3), 'salesVolume'].values
    m16_1_4 = train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 1), 'salesVolume'].values / \
              train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 4), 'salesVolume'].values
    m16_1_5 = train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 1), 'salesVolume'].values / \
              train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 5), 'salesVolume'].values

    m16_2_3 = train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 2), 'salesVolume'].values / \
              train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 3), 'salesVolume'].values
    m16_2_4 = train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 2), 'salesVolume'].values / \
              train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 4), 'salesVolume'].values
    m16_2_5 = train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 2), 'salesVolume'].values / \
              train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 5), 'salesVolume'].values
    m16_2_6 = train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 2), 'salesVolume'].values / \
              train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 6), 'salesVolume'].values

    m16_3_4 = train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 3), 'salesVolume'].values / \
              train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 4), 'salesVolume'].values
    m16_3_5 = train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 3), 'salesVolume'].values / \
              train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 5), 'salesVolume'].values
    m16_3_6 = train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 3), 'salesVolume'].values / \
              train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 6), 'salesVolume'].values
    m16_3_7 = train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 3), 'salesVolume'].values / \
              train_sales.loc[(train_sales.regYear == 2016) & (train_sales.regMonth == 7), 'salesVolume'].values

    m17_1_2 = train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 1), 'salesVolume'].values / \
              train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 2), 'salesVolume'].values
    m17_1_3 = train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 1), 'salesVolume'].values / \
              train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 3), 'salesVolume'].values
    m17_1_4 = train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 1), 'salesVolume'].values / \
              train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 4), 'salesVolume'].values
    m17_1_5 = train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 1), 'salesVolume'].values / \
              train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 5), 'salesVolume'].values

    m17_2_3 = train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 2), 'salesVolume'].values / \
              train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 3), 'salesVolume'].values
    m17_2_4 = train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 2), 'salesVolume'].values / \
              train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 4), 'salesVolume'].values
    m17_2_5 = train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 2), 'salesVolume'].values / \
              train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 5), 'salesVolume'].values
    m17_2_6 = train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 2), 'salesVolume'].values / \
              train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 6), 'salesVolume'].values

    m17_3_4 = train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 3), 'salesVolume'].values / \
              train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 4), 'salesVolume'].values
    m17_3_5 = train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 3), 'salesVolume'].values / \
              train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 5), 'salesVolume'].values
    m17_3_6 = train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 3), 'salesVolume'].values / \
              train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 6), 'salesVolume'].values
    m17_3_7 = train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 3), 'salesVolume'].values / \
              train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 7), 'salesVolume'].values

    m16_1 = m16_1_2 / 2 + m16_1_3 / 4 + m16_1_4 / 8 + m16_1_5 / 8
    m16_2 = m16_2_3 / 2 + m16_2_4 / 4 + m16_2_5 / 8 + m16_2_6 / 8
    m16_3 = m16_3_4 / 2 + m16_3_5 / 4 + m16_3_6 / 8 + m16_3_7 / 8

    m17_1 = m17_1_2 / 2 + m17_1_3 / 4 + m17_1_4 / 8 + m17_1_5 / 8
    m17_2 = m17_2_3 / 2 + m17_2_4 / 4 + m17_2_5 / 8 + m17_2_6 / 8
    m17_3 = m17_3_4 / 2 + m17_3_5 / 4 + m17_3_6 / 8 + m17_3_7 / 8

    m1 = m16_1 * 0.4 + m17_1 * 0.6
    m2 = m16_2 * 0.4 + m17_2 * 0.6
    m3 = m16_3 * 0.4 + m17_3 * 0.6

    evaluation_public.loc[evaluation_public.regMonth == 2, 'forecastVolum'] = train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 1), 'salesVolume'].values / m1
    evaluation_public.loc[evaluation_public.regMonth == 3, 'forecastVolum'] = train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 2), 'salesVolume'].values / m2
    evaluation_public.loc[evaluation_public.regMonth == 4, 'forecastVolum'] = train_sales.loc[(train_sales.regYear == 2017) & (train_sales.regMonth == 3), 'salesVolume'].values / m3
    return evaluation_public

rule2=yulaorule(train_sales.copy())

In [21]:
rule2.head(10)

,id,province,adcode,model,regYear,regMonth,forecastVolum
0,1,上海,310000,3c974920a76ac9c1,2018,1,9.312498
1,2,云南,530000,3c974920a76ac9c1,2018,1,12.045405
2,3,内蒙古,150000,3c974920a76ac9c1,2018,1,4.930692
3,4,北京,110000,3c974920a76ac9c1,2018,1,9.292539
4,5,四川,510000,3c974920a76ac9c1,2018,1,12.842968
5,6,安徽,340000,3c974920a76ac9c1,2018,1,6.194126
6,7,山东,370000,3c974920a76ac9c1,2018,1,14.105116
7,8,山西,140000,3c974920a76ac9c1,2018,1,6.675350
8,9,广东,440000,3c974920a76ac9c1,2018,1,76.486815
9,10,广西,450000,3c974920a76ac9c1,2018,1,11.187078


In [22]:
rule=rule2.merge(rule1,on=["adcode", "model", "province","regMonth"],how='left').rename(columns={'forecastVolum':'forecastVolum2'})
day_map = {1:31,2:28,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
rule['daycount']=rule['regMonth'].map(day_map)
rule['forecastVolum']=(0.5*rule['forecastVolum2']+0.5*rule['forecastVolum1'])*rule['daycount']

In [23]:
rule.head(10)

,id,province,adcode,model,regYear,regMonth,forecastVolum2,bodyType,forecastVolum1,daycount,forecastVolum
0,1,上海,310000,3c974920a76ac9c1,2018,1,9.312498,SUV,10.388013,31,305.357927
1,2,云南,530000,3c974920a76ac9c1,2018,1,12.045405,SUV,13.650471,31,398.286085
2,3,内蒙古,150000,3c974920a76ac9c1,2018,1,4.930692,SUV,5.400057,31,160.126608
3,4,北京,110000,3c974920a76ac9c1,2018,1,9.292539,SUV,17.412660,31,413.930582
4,5,四川,510000,3c974920a76ac9c1,2018,1,12.842968,SUV,14.100386,31,417.621975
5,6,安徽,340000,3c974920a76ac9c1,2018,1,6.194126,SUV,6.080924,31,190.263283
6,7,山东,370000,3c974920a76ac9c1,2018,1,14.105116,SUV,15.800246,31,463.533112
7,8,山西,140000,3c974920a76ac9c1,2018,1,6.675350,SUV,8.491733,31,235.089794
8,9,广东,440000,3c974920a76ac9c1,2018,1,76.486815,SUV,95.108915,31,2659.733816
9,10,广西,450000,3c974920a76ac9c1,2018,1,11.187078,SUV,13.960963,31,389.794634


In [24]:
rule.shape

(5280, 11)

In [25]:
rule['forecastVolum'] = rule['forecastVolum'].apply(lambda x: 0 if x < 0 else int(round(x)))
rule[['id','forecastVolum']].to_csv('guize1_0901.csv',index=False)

In [31]:
rule1 = pd.read_csv("guize1_0901.csv")
rule1.head(2)

,id,forecastVolum
0,1,305
1,2,398


In [32]:
rule2 = pd.read_csv("guize2_0901.csv")
rule2.head(2)

,id,forecastVolum
0,1,273
1,2,357


In [33]:
rule1["forecastVolum"] = rule1["forecastVolum"]*0.5 +rule2["forecastVolum"]*0.5
rule1.head(2)

,id,forecastVolum
0,1,289.0
1,2,377.5


In [34]:
rule1['forecastVolum'] = rule1['forecastVolum'].apply(lambda x: 0 if x < 0 else int(round(x)))
rule1[['id','forecastVolum']].to_csv('rongheguize_0901.csv',index=False)